In [2]:
import pandas as pd
import os

encoding="cp938"
def smellData(smellName):
    smellDF=pd.read_csv(os.path.join("ground truth",smellName+".csv"))
    smellDF=smellDF[["nomeSistema","class"]]
    smellDF[smellName]=1
    return smellDF

def getClassDF(folder):
    classDF=pd.read_csv(os.path.join(folder,"allClass.csv"))
    classDF["class"]=classDF["class"].str.replace("$",".")
    return classDF

def getMethodDF(folder):
    methodDF=pd.read_csv(os.path.join(folder,"allMethod.csv"))
    methodDF["class"]=methodDF["class"].str.replace("$",".")
    methodDF["qualifiedName"]=methodDF["method"].str.replace("/.*","",regex=True)
    methodDF["qualifiedName"]=methodDF["class"]+"."+methodDF["qualifiedName"]
    return methodDF

def getProjectBlocks(textFile):
    with open(textFile,"r") as f:
        projectString=f.read()
    projectBlocks=projectString.split("\n")
    return projectBlocks

def getProjectName(block):
    projectInfo=block.split(",")
    projectName=projectInfo[0]
    nomeSistema=projectInfo[3]
    return projectName,nomeSistema

def longMethod(methodDF,projectSmell):
    sortMethodDF=methodDF.sort_values("loc_x")
    drop=sortMethodDF.drop_duplicates(subset=["qualifiedName"],keep="last")
    smellName="LongMethod"
    projectSmellDF=projectSmell.drop(columns=["nomeSistema"])
    projectSmellDF=projectSmellDF.rename(columns={"class":"qualifiedName"})
    joinMetricsSmell=drop.merge(projectSmellDF,how="inner",on="qualifiedName")
    allMethod=methodDF.merge(joinMetricsSmell,how="left")
    allMethod[smellName]=allMethod[smellName].fillna(0)
    return allMethod

def featureEnvy(methodDF,projectSmell):
    sortMethodDF=methodDF.sort_values("ATFD_x")
    drop=sortMethodDF.drop_duplicates(subset=["qualifiedName"],keep="last")
    smellName="FeatureEnvy"
    projectSmellDF=projectSmell.drop(columns=["nomeSistema"])
    projectSmellDF=projectSmellDF.rename(columns={"class":"qualifiedName"})
    joinMetricsSmell=drop.merge(projectSmellDF,how="inner",on="qualifiedName")
    allMethod=methodDF.merge(joinMetricsSmell,how="left")
    allMethod[smellName]=allMethod[smellName].fillna(0)
    return allMethod

In [8]:
#クラススメルとの結合
smellName="RefusedBequest"

pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
pd.set_option("display.max_colwidth",None)
blocks=getProjectBlocks("version.txt")
for block in blocks:
    projectInfo=block.split(",")
    targetFolder=projectInfo[0]
    nomeSistema=projectInfo[3]
    

    dataFolder=os.path.join(os.getcwd(),"joinCKSPOON_4")
    folder=os.path.join(dataFolder,targetFolder)

    classDF=getClassDF(folder)
    smellDF=smellData(smellName)
    projectsSmellDF=smellDF.groupby("nomeSistema")
    print(nomeSistema)
    if nomeSistema in projectsSmellDF.groups.keys():
        projectSmellDF=projectsSmellDF.get_group(nomeSistema)
        projectSmellDF=projectSmellDF.drop(columns=["nomeSistema"])
        joinMetricsSmell=classDF.merge(projectSmellDF,how="left",on=["class"])
        joinMetricsSmell[smellName]=joinMetricsSmell[smellName].fillna(0)
        print("ground truth : ",len(projectSmellDF))
    else:
        joinMetricsSmell=classDF
        joinMetricsSmell[smellName]=0
        print("ground truth : 0")
        
    
    print("結合後smell数 : ",len(joinMetricsSmell[joinMetricsSmell[smellName]==1]))
    joinMetricsSmell.to_csv(os.path.join(folder,smellName+".csv"),index=False)
    print()

corenlp
ground truth :  22
結合後smell数 :  22

dbeaver
ground truth :  15
結合後smell数 :  15

guava
ground truth :  38
結合後smell数 :  38

elasticsearch-analysis
ground truth : 0
結合後smell数 :  0

hikari
ground truth : 0
結合後smell数 :  0

javafaker
ground truth :  1
結合後smell数 :  1

jedis
ground truth :  3
結合後smell数 :  3

jenkins
ground truth :  19
結合後smell数 :  19

jsoup
ground truth :  1
結合後smell数 :  1

junit4
ground truth :  2
結合後smell数 :  2

libgdx
ground truth :  81
結合後smell数 :  81

mockserver
ground truth :  4
結合後smell数 :  4

mybatis
ground truth :  2
結合後smell数 :  2

nanohttpd
ground truth : 0
結合後smell数 :  0

netty
ground truth : 0
結合後smell数 :  0

redisson
ground truth : 0
結合後smell数 :  0

retrofit
ground truth : 0
結合後smell数 :  0

shopizer
ground truth :  7
結合後smell数 :  7

spark
ground truth : 0
結合後smell数 :  0

webmagic
ground truth :  1
結合後smell数 :  1

xdman
ground truth :  1
結合後smell数 :  1

zookeeper
ground truth :  2
結合後smell数 :  1

ysoserial
ground truth : 0
結合後smell数 :  0

mapstruct
ground 

In [7]:
#メソッドスメルとの結合
smellName="FeatureEnvy"#"LongMethod"
blocks=getProjectBlocks("version.txt")
for block in blocks:
    projectName,nomeSistema=getProjectName(block)
    projectFolder=os.path.join("joinCKSPOON_4",projectName)
    methodDF=getMethodDF(projectFolder)
    smellDF=smellData(smellName)
    projectsSmellDF=smellDF.groupby("nomeSistema")
    if nomeSistema in projectsSmellDF.groups.keys():
        projectSmell=projectsSmellDF.get_group(nomeSistema)
        if smellName=="LongMethod":
            joinMetricsSmellDF=longMethod(methodDF,projectSmell)        
        elif smellName=="FeatureEnvy":
            joinMetricsSmellDF=featureEnvy(methodDF,projectSmell)
            
    else:
        joinMetricsSmell=methodDF
        joinMetricsSmell[smellName]=0
    joinMetricsSmellDF.to_csv(os.path.join(projectFolder,smellName+".csv"),index=False)
    
#LongMethodの場合locが最大のメソッドに対して，正解ラベルを付与
#FeatureEnvyの場合ATFDが最大のメソッドに対して，正解ラベルを付与


In [ ]:
smellName="FeatureEnvy"
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
pd.set_option("display.max_colwidth",None)
blocks=getProjectBlocks("version.txt")
for block in blocks:
    projectName,nomeSistema=getProjectName(block)
    projectFolder=os.path.join("joinCKSPOON_4",projectName)
    methodDF=getMethodDF(projectFolder)
    methodDF=methodDF[["qualifiedName","loc_x"]]
    smellDF=smellData(smellName)
    print(nomeSistema)
    projectsSmellDF=smellDF.groupby("nomeSistema")
    if nomeSistema in projectsSmellDF.groups.keys():
        projectSmell=projectsSmellDF.get_group(nomeSistema)
        projectSmell=projectSmell.rename(columns={"class":"qualifiedName"})
        join=methodDF.merge(projectSmell,how="inner")
        
        print("正解ラベル数 : ",len(projectSmell))
        print("正解ラベル重複数 : ",projectSmell.duplicated().sum())
        print("データ陽性数 : ",len(join[join[smellName]==1]))
        display(join[join["qualifiedName"].duplicated()].sort_values("qualifiedName"))
    print()